## This code is developed by Rick Chen

In [4]:
from platform import python_version

print(python_version())

3.6.8


In [5]:
import pandas as pd
import numpy as np
from scipy import sparse

from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, StandardScaler, OrdinalEncoder
#from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer

#from category_encoders import OrdinalEncoder

from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.calibration import CalibratedClassifierCV


from xgboost import XGBClassifier
import xgboost as xgb

from datetime import datetime

import re
import pickle

import nltk

from nltk.corpus import stopwords                   #Stopwords corpus
from nltk.tokenize import word_tokenize

import textblob
from textblob import TextBlob

from string import punctuation
## from spellchecker import SpellChecker

#from fnop.transforms import *

In [6]:
import types
def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__
list(imports())

['builtins',
 'builtins',
 'sklearn',
 'pandas',
 'numpy',
 'scipy.sparse',
 'sklearn.model_selection',
 'sklearn.preprocessing',
 'sklearn.linear_model',
 'sklearn.naive_bayes',
 'sklearn.metrics',
 'sklearn.svm',
 'xgboost',
 're',
 'pickle',
 'nltk',
 'textblob',
 'types']

In [7]:
dir0 = 'C:/Projects/FCSD/FNOP/old/'
data_name = "v0"

model_name = "v0"


In [8]:
data_train = pd.read_csv(dir0 + 'Traindata_pilot_sample_' + data_name + '.csv', engine='python')
data_test = pd.read_csv(dir0 + 'Testdata_pilot_sample_' + data_name + '.csv', engine='python')


In [9]:
print(data_train.shape)
print(data_test.shape)

df = data_train.groupby("target").size()
print (df.shape)

print("number of target in training data:", len(np.unique(data_train.target)))

print("number of target in testing data:", len(np.unique(data_test.target)))



(358000, 11)
(90005, 11)
(121,)
number of target in training data: 121
number of target in testing data: 121


In [10]:
print("number of vehicle line in training data:", len(np.unique(data_train.veh_line_cd)))

print("number of vehicle line in testing data:", len(np.unique(data_test.veh_line_cd)))

number of vehicle line in training data: 54
number of vehicle line in testing data: 52


In [11]:
print("number of engine in training data:", len(np.unique(data_train.eng_cd)))

print("number of engine in testing data:", len(np.unique(data_test.eng_cd)))

number of engine in training data: 96
number of engine in testing data: 90


In [12]:
# label encode the target variable 
le_yb = preprocessing.LabelEncoder()

le_yb.fit(data_train.target.values)

LabelEncoder()

In [13]:
np.unique(data_train.target)

array(['10346', '10V346', '11002', '1104', '1109', '1125', '1175', '1190',
       '11V002', '12029', '12A648', '13008', '13404', '13C788', '13E700',
       '13E754', '14529', '14547', '14A626', '14B205', '14B291', '14B504',
       '15200', '15603', '15K601', '16606', '17508', '17C476', '18125',
       '18808', '19703', '19712', '19805', '19835', '19860', '19972',
       '19980', '19A361', '19A498', '19C836', '19D596', '19D666',
       '19D742', '19D786', '19E616', '19G364', '19G365', '19G366',
       '19G367', '19G490', '19V649', '1A189', '1S175', '1S177', '1S190',
       '2041', '2078', '2225', '2234', '2261', '2321', '2386', '2552',
       '2553', '2A040', '2A225', '2A603', '2C326', '2C360', '2L527',
       '2V001', '2V120', '2V121', '2V321', '2V552', '2V553', '2V563',
       '3280', '3504', '3A197', '3A674', '3B436', '3V049', '3V050',
       '3V084', '3V085', '3V130', '3V131', '3V280', '3V504', '3V674',
       '5404788', '5K483', '6323208', '6323209', '6623208', '6623209',
       '6

In [14]:
print("test with a subset")

target_sample = ['10346', '10V346', '11002', '1104']

data_train =  data_train[data_train.target.isin(target_sample)]
data_test =  data_test[data_test.target.isin(target_sample)]

test with a subset


In [15]:
np.unique(data_test.eng_cd)

array(['C/H4', 'C/JT', 'C/KD', 'C/KP', 'C/KX', 'C/M1', 'C/MF', 'C/MG',
       'C/N3', 'C/N5', 'C/R9', 'C/RM', 'C/RN', 'C/S3', 'C/S7', 'C/SB',
       'C/SG', 'C/TA', 'C/TS', 'C/U4', 'C/UA', 'C/UN', 'C/VN', 'C/WQ',
       'C/WU', 'C/XQ', 'C/YL', 'C/YR', 'C/YV', 'C/YX', 'H/AL', 'H/AY',
       'H/AZ', 'H/BU', 'H/CH', 'H/CK', 'H/HV', 'H/HW', 'H/HY', 'H/WB',
       'H/WC', 'T/GY', 'T/H4', 'T/JT', 'T/KP', 'T/KW', 'T/KX', 'T/KZ',
       'T/M9', 'T/MF', 'T/N3', 'T/N5', 'T/NE', 'T/R4', 'T/R9', 'T/RAV',
       'T/RAX', 'T/RBF', 'T/RM', 'T/S3', 'T/S7', 'T/SA', 'T/SG', 'T/T6',
       'T/TA', 'T/TP', 'T/TR', 'T/UR', 'T/VN', 'T/VZ', 'T/WA', 'T/WQ',
       'T/WX', 'T/YE', 'T/YF', 'T/YH'], dtype=object)

In [16]:
X_train = data_train[['cust_txt', 'tis_wsd', 'milge', 'veh_line_cd', 'mdl_yr', 'eng_cd']]
X_test = data_test[['cust_txt', 'tis_wsd', 'milge', 'veh_line_cd', 'mdl_yr', 'eng_cd']]

y_train = le_yb.transform(data_train.target.values)
y_test = le_yb.transform(data_test.target.values)

In [17]:
X_train.dtypes

cust_txt        object
tis_wsd          int64
milge          float64
veh_line_cd     object
mdl_yr           int64
eng_cd          object
dtype: object

In [18]:
pickle.dump(le_yb, open('encoder_' + model_name + '.pkl', 'wb'))

print(X_train.head())

                                            cust_txt  tis_wsd    milge  \
0         CS NOISE ENGINE AREA   POSSIBLE ALTERNATOR       13  14776.0   
1  CUSTOMER STATES THERE IS A WHINING NOISE IN TH...       15  15146.0   
2  CHECK BATTERY LIGHT IS COMING ON WHILE DRIVING...       13  11198.0   
3  TOWED IN....BATTERY LIGHT, A.B.S. LIGHT, RADIO...        0    731.0   
4                                     NO START/TOWIN       22   4471.0   

  veh_line_cd  mdl_yr eng_cd  
0        C/AU    2011   C/H4  
1        C/AU    2011   C/H4  
2        C/AU    2011   C/H4  
3        C/AU    2011   C/H4  
4        C/AU    2011   C/H4  


In [15]:
######## create transformation

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

    
class MyLabelBinarizer(BaseEstimator, TransformerMixin):

    def __init__(self, *args, **kwargs):

        self.encoder = LabelBinarizer(*args, **kwargs)

    def fit(self, x, y=0):

        self.encoder.fit(x)

        return self

    def transform(self, x, y=0):

        return self.encoder.transform(x)

class TextProcessor(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        # Import synonyms table for lookup
        #lookup_table = '/s/ychen310/SM-FNOP/Data/Misspellings_Correction_Synonyms_Lookup_Table_Master.xlsx'
        lookup_table = 'Misspellings_Correction_Synonyms_Lookup_Table_Master.xlsx'

        syn_df=pd.read_excel(lookup_table)

        syn_df = syn_df[syn_df['Review'] != "D"]

        syn_dict=dict(zip(syn_df.misspelled, syn_df.corrected))
        
        def lemmatize_with_postag(sentence):
            sent = TextBlob(sentence)
            tag_dict = {"J":'a',
                               "N":'n',
                               "V":'v',
                               "R":'r'}
            words_and_tags = [(w, tag_dict.get(pos[0],'n')) for w, pos in sent.tags]
            lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
            return " ".join(lemmatized_list)
        
        def TextPreprocess(text):
            # Avoid mixed data type errors and apply str on all rows

            text = str(text).lower()

            #Remove HTML Tags
            text = re.sub('<[^>]*>', '', text)
            r1 = re.compile('<.*?>')
            text = re.sub(r1, ' ', text)
            # Remove any token with numbers and/or special characters

            r2 = re.compile(r'^[0-9\W\_]+$')
            text = re.sub(r2, ' ', text)

            #Removing Punctuations
            text = re.sub(r'[?|!|\'|"|#]',r'',text)
            text = re.sub(r'[.|,|)|(|\|/]',r' ',text)        

            # Tokenize all text data using nltk package

            text = word_tokenize(text)

            # Remove stop words
            stop = stopwords.words('english')
            stop1 = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','c/s','c/states','c/stated','cu', 'cs','c-s', 'cus','tomer', 'stomer', 'tate', 'cust',  'cust states', 'advise','nana','NaN','customer','states','rr','nan','wont','ck','check','pls','please', 'client','owner']

            # Combine default nltk stop list and custom stop list to generate a deduplicated list
            english_stops = list(set(stop+stop1))

            text = [item for item in text if item not in english_stops]

            # Remove special characters occuring as prefixes
            text = [re.sub(r'^[\W\_]+','',str(i)) for i in text]
            # Remove special characters occuring as suffixes

            text = [re.sub(r'[\W\_]+$','',str(i)) for i in text]

            # Replace special characters with spaces when occuring within the token 
            #(i.e. not at the beginning or the end) only for tokens greater than 5 characters in length

            text =[re.sub(r'[\W\_]+', ' ', str(i)) if len(str(i)) > 5 else i for i in text]

            # Replace special characters with null when occuring within the token 
            #(i.e. not at the beginning or the end) only for tokens less than 6 characters in length

            text = [re.sub(r'[\W\_]+', '', str(i)) if len(str(i)) < 6 else i for i in text]
            # Remove stop words after data cleansing steps to remove any residual stop words

            text = [item for item in text if item not in english_stops]

            # remove any numbers after data cleansing steps to remove any residual numeric values

            r3 = re.compile(r'^[0-9\W\_]+$')
            text = [i for i in text if not r3.match(i)]

            #Perform synonym lookup to find matching correctly spelled tokens for misspelled tokens
            #spell=SpellChecker() 

            text = [syn_dict.get(str(i), str(i)) for i in text]

            # Convert the list to string before passing to lemmatization module
            text = ' '.join(text)

            # Perform Lemmatization on the cleaned up data before tfidf vectorization

            text = lemmatize_with_postag(text)

            return(text)
        
        pd_s = pd.Series([TextPreprocess(x) for x in X[self.field]])
        return pd_s #X[self.field]

class JoinReliabilityDataTransform(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass        
        
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
        
     #   data_path = '/s/ychen310/SM-FNOP/Data/f_rate_WB_118month_poc3.csv'
        data_path = 'f_rate_WB_118month_poc3.csv'

        reliability_df = pd.read_csv(data_path, engine='python')
        reliability_df.columns = ['prt_num', 'veh_line_cd', 'mdl_yr', 'tis_wsd', 'Avg_Weibull_Failure', 'f_rate']
        
        
        data_reliab_pivot = reliability_df.pivot_table(index=['veh_line_cd', 'mdl_yr', 'tis_wsd'], columns='prt_num', values=['Avg_Weibull_Failure'], aggfunc='first')
        
        reliability_df_t = data_reliab_pivot.fillna(0.0)
        
        print(reliability_df_t.shape)
        
        df1 = pd.merge(X[['tis_wsd', 'veh_line_cd', 'mdl_yr']], reliability_df_t, how='left', on=['veh_line_cd', 'mdl_yr', 'tis_wsd'])
        
        
        df1 = df1.drop(['tis_wsd', 'veh_line_cd', 'mdl_yr'], axis=1)
        
        return df1


In [16]:
## Build sklearn pipeline
print("start building sklearn pipeline")

numeric_features = ['milge', 'tis_wsd']

categorical_features = ['veh_line_cd','mdl_yr', 'eng_cd']


start building sklearn pipeline


In [17]:
# set vectorizing parameters
max_words_param = 20000
max_df_param = 0.95
min_df_param = 3
stops_list = stopwords.words('english')
    
    
num_pipeline = Pipeline([('selector', DataFrameSelector(numeric_features)),
                        ('standard',StandardScaler())])

cat_pipeline = Pipeline([('selector', DataFrameSelector(categorical_features)),
#                        ('Oridnal_Encoder', OrdinalEncoder())])
                       ('oneHot',OneHotEncoder(handle_unknown='ignore'))])  
# note that, handle-unknown is needed to avoid unknown category error

txt_pipeline = Pipeline([
            ('textpreprocessor', TextProcessor('cust_txt')),
            ('tfidf', TfidfVectorizer(stop_words=stops_list,
                     min_df=min_df_param, max_df=max_df_param, ngram_range=(1,3), max_features = 600)),
#            ('svd', TruncatedSVD(algorithm='randomized', n_components=300)), #for XGB
        ])

rel_pipeline = Pipeline([
           ('Join_Reliability_Data', JoinReliabilityDataTransform()),
        ])



In [18]:

# set XGboost prameters
param = {
    'max_depth': 5,  # the maximum depth of each tree
    'eta': 0.1,  # the training step for each iteration
    'subsample': 0.6, # ratio of training data randomly sampled in each iteration, default = 1 or 100%
    'objective': 'multi:softprob',  # softprob will result in a probability for each class, unlike softmax
 #   'num_class': num_y_levels, # the number of classes that exist in this datset
    'seed': 42, # set to particular value to allow for reproducible results
    'eval_metric': 'merror',
    'verbose': 2,
    'tree_method': 'gpu_hist',
    'n_jobs': -1,
    'n_estimators' : 300
}


In [19]:
classifier = Pipeline([
    ('feature', FeatureUnion(transformer_list=[
    
        ('rel_pipeline', rel_pipeline),
        ('txt_pipeline', txt_pipeline),    
        ('num_pipeline', num_pipeline),
        ('cat_pipeline', cat_pipeline)])),
       
    ('clf', XGBClassifier(param=param))
    ])


In [20]:
startTime=datetime.now()

classifier.fit(data_train, y_train)



(24426, 111)


C:\Users\JLUO51\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


Pipeline(steps=[('feature',
                 FeatureUnion(transformer_list=[('rel_pipeline',
                                                 Pipeline(steps=[('Join_Reliability_Data',
                                                                  JoinReliabilityDataTransform())])),
                                                ('txt_pipeline',
                                                 Pipeline(steps=[('textpreprocessor',
                                                                  TextProcessor(field='cust_txt')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer(max_df=0.95,
                                                                                  max_features=600,
                                                                                  min_df=3,
                                                                                  ngram_range=(1,
   

In [21]:
print("run testing")
preds = classifier.predict(X_test)

print(datetime.now() - startTime)


run testing
(24426, 111)


C:\Users\JLUO51\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


0:02:05.804152


In [22]:

from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix
print ("Accuracy:", accuracy_score(y_test, preds))



Accuracy: 0.932783608195902


In [23]:
print('Test training for all the target variables')
pickle.dump(classifier, open('classifier_' + model_name + '.pkl', 'wb'))

Test training for all the target variables
